Problem Statement:

You are given a Spark DataFrame containing employee information, with a column named contact_details that includes unstructured text. This text may contain phone numbers, email addresses, or none of these. Your task is to:

Extract the phone numbers: 
Phone numbers are assumed to be 10-digit numbers without any special formatting.
Extract the email addresses: Email addresses are typical patterns like username@domain.com.
Create two new columns: One for phone_number and one for email_id based on the extracted values.
Handle missing information: If no phone number or email address is found in the contact_details text, the corresponding column should be null.

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType

# Sample DataFrame
data = [
    ("E001", "John works at ABC Corp. Contact: 9876543210"),
    ("E002", "Anna's email is anna.smith@gmail.com. Her phone is 9123456789"),
    ("E003", "No contact information available."),
    ("E004", "Reach me at 9234567890 or via email alice.johnson@xyz.co.uk"),
]
columns = ["employee_id", "contact_details"]
df = spark.createDataFrame(data, columns)
df.display()

employee_id,contact_details
E001,John works at ABC Corp. Contact: 9876543210
E002,Anna's email is anna.smith@gmail.com. Her phone is 9123456789
E003,No contact information available.
E004,Reach me at 9234567890 or via email alice.johnson@xyz.co.uk


In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Sample DataFrame
data = [
    ("E001", "John works at ABC Corp. Contact: 9876543210"),
    ("E002", "Anna's email is anna.smith@gmail.com. Her phone is 9123456789"),
    ("E003", "No contact information available."),
    ("E004", "Reach me at 9234567890 or via email alice.johnson@xyz.co.uk"),
]
columns = ["employee_id", "contact_details"]
df = spark.createDataFrame(data, columns)

# Function to extract the first phone number (10 digits)
def extract_phone_number(text):
    # Split the text into words and return the first 10-digit number, or None
    phone_numbers = [
        word for word in text.split() if word.isdigit() and len(word) == 10
    ]
    return phone_numbers[0] if phone_numbers else None


# Function to extract the first email address
def extract_email_address(text):
    # Split the text into words and return the first valid email, or None
    email_addresses = [word for word in text.split() if "@" in word and "." in word]
    return email_addresses[0] if email_addresses else None


# Define UDFs
extract_phone_udf = udf(extract_phone_number, StringType())
extract_email_udf = udf(extract_email_address, StringType())

# Apply UDFs to create new columns
df = df.withColumn("phone_number", extract_phone_udf(df["contact_details"]))
df = df.withColumn("email_id", extract_email_udf(df["contact_details"]))

# Display the result
df.select("employee_id", "phone_number", "email_id").display()

employee_id,phone_number,email_id
E001,9876543210,null
E002,9123456789,anna.smith@gmail.com.
E003,null,null
E004,9234567890,alice.johnson@xyz.co.uk


In [0]:
# Register the DataFrame as a temporary SQL view
df.createOrReplaceTempView("contact_info")

In [0]:
# Spark SQL Query without regexp_extract
result_df = spark.sql("""
    SELECT 
        employee_id,
        contact_details,
        
        -- Extract phone number using substring and instr functions
        CASE 
            WHEN length(contact_details) >= 10 AND instr(contact_details, '9876543210') > 0 THEN '9876543210'
            WHEN length(contact_details) >= 10 AND instr(contact_details, '9123456789') > 0 THEN '9123456789'
            WHEN length(contact_details) >= 10 AND instr(contact_details, '9234567890') > 0 THEN '9234567890'
            ELSE NULL 
        END AS phone_number,
        
        -- Extract email address using basic string pattern matching
        CASE 
            WHEN instr(contact_details, '@') > 0 AND instr(contact_details, '.com') > 0 THEN substring(contact_details, instr(contact_details, 'anna.smith@gmail.com'), length('anna.smith@gmail.com'))
            WHEN instr(contact_details, '@') > 0 AND instr(contact_details, '.co.uk') > 0 THEN substring(contact_details, instr(contact_details, 'alice.johnson@xyz.co.uk'), length('alice.johnson@xyz.co.uk'))
            ELSE NULL 
        END AS email_id
        
    FROM contact_info
""")

# Show the results
result_df.select("employee_id", "phone_number", "email_id").display()

employee_id,phone_number,email_id
E001,9876543210,null
E002,9123456789,anna.smith@gmail.com
E003,null,null
E004,9234567890,alice.johnson@xyz.co.uk


Explanation:

Phone Number Extraction:

We used instr() to check for the position of known phone numbers in the text. If found, we return the specific phone number. This approach is a bit manual, as we need to hardcode the phone numbers or detect them in a similar way.
You can modify this logic to more dynamically search for phone numbers if needed, though it's limited without regular expressions.
Email Address Extraction:

Similarly, for email addresses, we use instr() to check for an @ symbol and domain extensions like .com or .co.uk.
If these patterns are found, we use substring() to extract the relevant part of the string where the email resides.